<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Clearing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "git+https://github.com/Karasiari/HuGraphFinal.git"

  Cloning https://github.com/Karasiari/HuGraphFinal.git to /tmp/pip-req-build-0c1411nc
  Running command git clone --filter=blob:none --quiet https://github.com/Karasiari/HuGraphFinal.git /tmp/pip-req-build-0c1411nc
  Resolved https://github.com/Karasiari/HuGraphFinal.git to commit acf0d0ffab1614e8deab011e669f39b3de5bff0f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hugraphfin: filename=hugraphfin-0.1.0-py3-none-any.whl size=31397 sha256=820c7ade205ac4d2f7e0095b91bd06e66a2aa58f983f46ca6af6345ab81e23d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-0zhdbpi4/wheels/46/83/89/572a6db8e6320eb877bf6ced236fa61f993538a12c5e43c72e
Successfully built hugraphfin


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import os
from google.colab import files

# две функции - читалки hu графов в мультиграфы

BITRATE_DENOMINATOR = 100

def hu_csv_to_graphs(path, demands_path, capacity_path):
  Network = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  Traffic = pd.read_csv(demands_path, header=None, names = ['id', 'source', 'target', 'bitrate'])
  Capacity = pd.read_csv(capacity_path, header=None, names = ['name', 'value'])
  network = Network.iloc[1:].copy()
  traffic = Traffic.iloc[1:].copy()
  capacity = Capacity.iloc[1:].copy()

  network['source'] = network['source'].astype(int)
  network['target'] = network['target'].astype(int)
  traffic['source'] = traffic['source'].astype(int)
  traffic['target'] = traffic['target'].astype(int)
  traffic['bitrate'] = traffic['bitrate'].astype(float) / BITRATE_DENOMINATOR
  capacity_value = float(capacity[capacity['name']=='NumberOfWavelengths']['value'])

  sources = network['source'].tolist()
  targets = network['target'].tolist()
  sources_traffic = traffic['source'].tolist()
  targets_traffic = traffic['target'].tolist()
  bitrates_traffic = traffic['bitrate'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_edges = []
  traffic_edges = []

  for source, target in zip(sources, targets):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_edges.append((new_source, new_target, {"capacity": capacity_value}))
  for source, target, bitrate in zip(sources_traffic, targets_traffic, bitrates_traffic):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    traffic_edges.append((new_source, new_target, {"weight": bitrate}))

  adj_graph = nx.MultiGraph()
  adj_graph.add_edges_from(adj_edges)
  traffic_graph = nx.MultiDiGraph()
  traffic_graph.add_nodes_from(range(adj_graph.number_of_nodes()))
  traffic_graph.add_edges_from(traffic_edges)

  return adj_graph, traffic_graph, capacity_value

def get_hu_graphs(base_path, specified_graphs, specified):
  Graphs = {}
  csv_tables = []
  for folder_name in os.listdir(base_path):
    if folder_name in specified_graphs or not specified:
      folder_path = os.path.join(base_path, folder_name)
      csv_path = os.path.join(folder_path, 'links.csv')
      csv_path_demands = os.path.join(folder_path, 'demands.csv')
      csv_path_capacity = os.path.join(folder_path, 'params.csv')
      csv_tables.append(folder_name)

      adj_graph, traffic_graph, capacity_value = hu_csv_to_graphs(csv_path, csv_path_demands, csv_path_capacity)
      Graphs[folder_name] = {'adj_graph': adj_graph, 'traffic_graph': traffic_graph, 'capacity_value': capacity_value}

  return Graphs

path_to_folder = '/content/drive/MyDrive/Кола'

In [3]:
hu_graphs_names = ['cola_t2']#, 'cola_t3']
hu_graphs = get_hu_graphs(path_to_folder, hu_graphs_names, True)

/tmp/ipython-input-3554038479.py:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  capacity_value = float(capacity[capacity['name']=='NumberOfWavelengths']['value'])


In [ ]:
from graph_exps import HuGraphForExps
from graph_exps.exp import *
available_volumes = ((1, 0.9), (2, 0.1))

for name, graph in hu_graphs.items():
  Graph = HuGraphForExps(graph['adj_graph'], graph['traffic_graph'])
  additional_resources = [80.0, 80.0, 80.0]
  allocation_types = ['alpha', 'random_alpha', 'random']
  results = expand_test_for_graph(Graph, additional_resources, allocation_types, tries_for_allocation=5, epsilon=1.2, available_volumes=available_volumes)

In [5]:
results

allocation solved rerouted volume ratio  \
alpha        1              True                  1.18   
             2              True                  1.18   
             3              True                  1.17   
             4              True                  1.18   
             5              True                  1.18   
random_alpha 1              True                  1.18   
             2              True                  1.18   
             3              True                  1.18   
             4              True                  1.17   
             5              True                  1.18   
random       1              True                  1.17   
             2              True                  1.18   
             3              True                  1.18   
             4              True                  1.18   
             5              True                  1.17   

               gamma for failed (0, 1) gamma for failed (0, 2)  \
alpha        1                   33.83                    9.75   
             2                   33.83                    9.75   
             3                   33.83                    9.75   
             4                   33.83                    9.75   
             5                   33.83                    9.75   
random_alpha 1                    71.0                    8.78   
             2                    71.0                    8.78   
             3                    71.0                    8.78   
             4                    71.0                    8.78   
             5                    71.0                    8.78   
random       1                    40.4                   10.33   
             2                    40.4                   10.33   
             3                    40.4                   10.33   
             4                    40.4                   10.33   
             5                    40.4                   10.33   

               gamma for failed (0, 18) gamma for failed (1, 2)  \
alpha        1                    138.0                    9.57   
             2                    138.0                    9.57   
             3                    138.0                    9.57   
             4                    138.0                    9.57   
             5                    138.0                    9.57   
random_alpha 1                    140.0                   11.65   
             2                    140.0                   11.65   
             3                    140.0                   11.65   
             4                    140.0                   11.65   
             5                    140.0                   11.65   
random       1                    46.67                    9.57   
             2                    46.67                    9.57   
             3                    46.67                    9.57   
             4                    46.67                    9.57   
             5                    46.67                    9.57   

               gamma for failed (1, 8) gamma for failed (1, 22)  \
alpha        1                   15.07                    14.13   
             2                   15.07                    14.13   
             3                   15.07                    14.13   
             4                   15.07                    14.13   
             5                   15.07                    14.13   
random_alpha 1                   13.47                    31.78   
             2                   13.47                    31.78   
             3                   13.47                    31.78   
             4                   13.47                    31.78   
             5                   13.47                    31.78   
random       1                   12.59                    18.91   
             2                   12.59                    18.91   
             3                   12.59                    18.91   
             4    